# 可选实验 - Softmax 函数
在本实验中，我们将探索 softmax 函数。该函数在 Softmax 回归和神经网络中解决多分类问题时都会使用。  

<center>  <img  src="./images/C2_W2_Softmax_Header.PNG" width="600" />  <center/>

  

In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from IPython.display import display, Markdown, Latex
from sklearn.datasets import make_blobs
%matplotlib widget
from matplotlib.widgets import Slider
from lab_utils_common import dlc
from lab_utils_softmax import plt_softmax
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

> **注意**：通常，在本课程中，notebook 使用从 0 开始到 N-1 结束的计数约定，$\sum_{i=0}^{N-1}$，而讲座从 1 开始到 N 结束，$\sum_{i=1}^{N}$。这是因为代码通常从 0 开始迭代，而在讲座中，从 1 到 N 计数会导致更简洁、更简洁的方程。本 notebook 的方程比典型的实验更多，因此将打破约定，将从 1 计数到 N。

## Softmax 函数
在 softmax 回归和具有 Softmax 输出的神经网络中，都会生成 N 个输出，并选择一个输出作为预测类别。在这两种情况下，向量 $\mathbf{z}$ 由线性函数生成，然后应用于 softmax 函数。Softmax 函数将 $\mathbf{z}$ 转换为如下所述的概率分布。应用 softmax 后，每个输出将在 0 和 1 之间，并且输出将相加为 1，因此可以将它们解释为概率。较大的输入将对应于较大的输出概率。
<center>  <img  src="./images/C2_W2_SoftmaxReg_NN.png" width="600" />  

The softmax function can be written:
$$a_j = \frac{e^{z_j}}{ \sum_{k=1}^{N}{e^{z_k} }} \tag{1}$$
The output $\mathbf{a}$ is a vector of length N, so for softmax regression, you could also write:
\begin{align}
\mathbf{a}(x) =
\begin{bmatrix}
P(y = 1 | \mathbf{x}; \mathbf{w},b) \\
\vdots \\
P(y = N | \mathbf{x}; \mathbf{w},b)
\end{bmatrix}
=
\frac{1}{ \sum_{k=1}^{N}{e^{z_k} }}
\begin{bmatrix}
e^{z_1} \\
\vdots \\
e^{z_{N}} \\
\end{bmatrix} \tag{2}
\end{align}


这表明输出是一个概率向量。第一个条目是在给定输入 $\mathbf{x}$ 和参数 $\mathbf{w}$ 和 $\mathbf{b}$ 的情况下，输入是第一个类别的概率。  
让我们创建一个 NumPy 实现：

In [2]:
def my_softmax(z):
    ez = np.exp(z)              #element-wise exponenial
    sm = ez/np.sum(ez)
    return(sm)

下面，使用滑块改变 `z` 输入的值。

In [3]:
plt.close("all")
plt_softmax(my_softmax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

当您改变上面 z 的值时，需要注意以下几点：
* softmax 分子中的指数放大了值之间的微小差异
* 输出值相加为一
* softmax 跨越所有输出。例如，`z0` 的变化将改变 `a0`-`a3` 的值。将其与 ReLU 或 Sigmoid 等其他激活函数进行比较，这些激活函数具有单个输入和单个输出。

## Cost
<center> <img  src="./images/C2_W2_SoftMaxCost.png" width="400" />    <center/>

与 Softmax 相关的损失函数，即交叉熵损失，是：
\begin{equation}
  L(\mathbf{a},y)=\begin{cases}
    -log(a_1), & \text{如果 $y=1$}.\\
        &\vdots\\
     -log(a_N), & \text{如果 $y=N$}
  \end{cases} \tag{3}
\end{equation}

其中 y 是此示例的目标类别，$\mathbf{a}$ 是 softmax 函数的输出。特别是，$\mathbf{a}$ 中的值是相加为一的概率。
>**回顾：**在本课程中，损失是针对一个示例的，而代价涵盖所有示例。 
 
 
请注意，在上面的 (3) 中，只有对应于目标的行对损失有贡献，其他行为零。要编写代价方程，我们需要一个"指示函数"，当索引与目标匹配时为 1，否则为零。 
    $$\mathbf{1}\{y == n\} = =\begin{cases}
    1, & \text{如果 $y==n$}.\\
    0, & \text{否则}.
  \end{cases}$$
现在代价是：
\begin{align}
J(\mathbf{w},b) = -\frac{1}{m} \left[ \sum_{i=1}^{m} \sum_{j=1}^{N}  1\left\{y^{(i)} == j\right\} \log \frac{e^{z^{(i)}_j}}{\sum_{k=1}^N e^{z^{(i)}_k} }\right] \tag{4}
\end{align}

其中 $m$ 是示例数量，$N$ 是输出数量。这是所有损失的平均值。


## Tensorflow
本实验将讨论在 Tensorflow 中实现 softmax、交叉熵损失的两种方法："明显"方法和"首选"方法。前者是最直接的，而后者在数值上更稳定。

让我们首先创建一个数据集来训练多分类模型。

In [4]:
# make  dataset for example
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
X_train, y_train = make_blobs(n_samples=2000, centers=centers, cluster_std=1.0,random_state=30)

### *明显*的组织方式

下面的模型在最后一个 Dense 层中将 softmax 作为激活函数实现。
损失函数在 `compile` 指令中单独指定。 

损失函数是 `SparseCategoricalCrossentropy`。此损失在上面 (3) 中描述。在此模型中，softmax 在最后一层进行。损失函数接受 softmax 输出，这是一个概率向量。 

In [5]:
model = Sequential(
    [ 
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(4, activation = 'softmax')    # < softmax activation here
    ]
)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    X_train,y_train,
    epochs=10
)
        

Epoch 1/10
63/63 [==============================] - 0s 966us/step - loss: 0.8312
Epoch 2/10
63/63 [==============================] - 0s 1ms/step - loss: 0.3203
Epoch 3/10
63/63 [==============================] - 0s 1ms/step - loss: 0.1408
Epoch 4/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0847
Epoch 5/10
63/63 [==============================] - 0s 944us/step - loss: 0.0626
Epoch 6/10
63/63 [==============================] - 0s 974us/step - loss: 0.0515
Epoch 7/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0447
Epoch 8/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0402
Epoch 9/10
63/63 [==============================] - 0s 922us/step - loss: 0.0361
Epoch 10/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0332


因为 softmax 集成到输出层中，所以输出是一个概率向量。

In [6]:
p_nonpreferred = model.predict(X_train)
print(p_nonpreferred [:2])
print("largest value", np.max(p_nonpreferred), "smallest value", np.min(p_nonpreferred))

[[3.39e-03 1.02e-02 9.73e-01 1.30e-02]
 [9.97e-01 3.46e-03 9.35e-06 9.02e-06]]
largest value 0.9999994 smallest value 4.1378247e-09


### 首选 <img align="Right" src="./images/C2_W2_softmax_accurate.png"  style=" width:400px; padding: 10px 20px ; ">
回想一下讲座，如果在训练期间将 softmax 和损失结合起来，可以获得更稳定和准确的结果。这里显示的"首选"组织方式可以实现这一点。


在首选组织中，最后一层具有线性激活。由于历史原因，这种形式的输出被称为 *logits*。损失函数有一个附加参数：`from_logits = True`。这告诉损失函数 softmax 操作应该包含在损失计算中。这允许优化的实现。

In [7]:
preferred_model = Sequential(
    [ 
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(4, activation = 'linear')   #<-- Note
    ]
)
preferred_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  #<-- Note
    optimizer=tf.keras.optimizers.Adam(0.001),
)

preferred_model.fit(
    X_train,y_train,
    epochs=10
)
        

Epoch 1/10
63/63 [==============================] - 0s 1ms/step - loss: 1.0936
Epoch 2/10
63/63 [==============================] - 0s 1ms/step - loss: 0.4933
Epoch 3/10
63/63 [==============================] - 0s 1ms/step - loss: 0.2809
Epoch 4/10
63/63 [==============================] - 0s 1ms/step - loss: 0.1506
Epoch 5/10
63/63 [==============================] - 0s 966us/step - loss: 0.0916
Epoch 6/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0694
Epoch 7/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0592
Epoch 8/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0527
Epoch 9/10
63/63 [==============================] - 0s 919us/step - loss: 0.0489
Epoch 10/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0457


#### 输出处理
请注意，在首选模型中，输出不是概率，而是可以从大的负数到大的正数。在执行期望概率的预测时，必须通过 softmax 发送输出。 
让我们看看首选模型的输出：

In [8]:
p_preferred = preferred_model.predict(X_train)
print(f"two example output vectors:\n {p_preferred[:2]}")
print("largest value", np.max(p_preferred), "smallest value", np.min(p_preferred))

two example output vectors:
 [[-2.72 -4.45  2.9  -1.37]
 [ 6.42  1.32 -1.32 -6.74]]
largest value 12.410254 smallest value -13.030992


输出预测不是概率！
如果期望的输出是概率，则输出应该通过 [softmax](https://www.tensorflow.org/api_docs/python/tf/nn/softmax) 处理。

In [9]:
sm_preferred = tf.nn.softmax(p_preferred).numpy()
print(f"two example output vectors:\n {sm_preferred[:2]}")
print("largest value", np.max(sm_preferred), "smallest value", np.min(sm_preferred))

two example output vectors:
 [[3.55e-03 6.34e-04 9.82e-01 1.38e-02]
 [9.94e-01 6.05e-03 4.35e-04 1.92e-06]]
largest value 0.9999995 smallest value 1.5196424e-11


要选择最可能的类别，不需要 softmax。可以使用 [np.argmax()](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) 找到最大输出的索引。

In [10]:
for i in range(5):
    print( f"{p_preferred[i]}, category: {np.argmax(p_preferred[i])}")

[-2.72 -4.45  2.9  -1.37], category: 2
[ 6.42  1.32 -1.32 -6.74], category: 0
[ 4.64  1.5  -1.31 -5.3 ], category: 0
[-0.29  3.76 -3.11 -1.58], category: 1
[-0.64 -6.05  5.23 -5.2 ], category: 2


## SparseCategorialCrossentropy 或 CategoricalCrossEntropy
Tensorflow 有两种可能的目标值格式，损失函数的选择定义了期望的格式。
- SparseCategorialCrossentropy：期望目标是对应于索引的整数。例如，如果有 10 个可能的目标值，y 将在 0 和 9 之间。 
- CategoricalCrossEntropy：期望示例的目标值是 one-hot 编码，其中目标索引处的值为 1，而其他 N-1 个条目为零。一个有 10 个可能目标值的示例，其中目标是 2，将是 [0,0,1,0,0,0,0,0,0,0]。


## 恭喜！
在本实验中，您
- 更加熟悉 softmax 函数及其在 softmax 回归和神经网络中 softmax 激活的使用。 
- 学习了 Tensorflow 中的首选模型构建：
    - 最后一层没有激活（与线性激活相同）
    - SparseCategoricalCrossentropy 损失函数
    - 使用 from_logits=True
- 认识到与 ReLU 和 Sigmoid 不同，softmax 跨越多个输出。